## Basic implementation of Google Gemini

In [4]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

In [7]:


message = llm.invoke("What is the capital of pakistan")
print(message.content)

The capital of Pakistan is Islamabad.


## Langchain PromptTemplates

In [26]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is the this given language: {content}, Give only one word answer, based on the language only. For Example User Input: test = 100, Output: Javascript")

print(llm.invoke(prompt.format(content = "#include <stdlib.h>")).content)

C


### Langchain Chains

In [27]:
from langchain.chains import LLMChain

llm_chain = LLMChain(llm = llm, prompt = prompt)

print(llm_chain.run("from langchain.prompts import PromptTemplate"))

Python


#### Multiple variables chaining

In [30]:
complex_prompt = PromptTemplate.from_template("What is the capital of the {country} and the currency of the {country}, The city must be {city}")
complex_llm_chain = LLMChain(llm = llm, prompt = complex_prompt)

In [33]:
print(complex_llm_chain.run(country = "India" , city = "Karachi"))

The capital of India is New Delhi, not Karachi.  Karachi is a major city in Pakistan.

The currency of India is the Indian Rupee (INR).


### Sequential Prompt Template

In [48]:
from langchain.chains import SimpleSequentialChain


prompt1 = PromptTemplate.from_template("What is this language: {content}, Give only one word answer, based on the language only. For Example User Input: test = 100, Output: Javascript")

chain1 = LLMChain(llm = llm, prompt = prompt1 , output_key = "language")

prompt2 = PromptTemplate.from_template("Based on the {language} that was found, State few use cases of that language.")
chain2 = LLMChain(llm = llm, prompt = prompt2 , output_key = "use_cases") 

sChain = SimpleSequentialChain(chains=[chain1, chain2])

print(sChain.run("from langchain.prompts import PromptTemplate"))

Python's versatility leads to a wide range of use cases across various domains. Here are a few prominent examples:

**Web Development:**

* **Building web applications:** Frameworks like Django and Flask simplify the creation of robust and scalable web applications, handling backend logic, database interactions, and user authentication.  Instagram, Spotify, and Pinterest utilize Python for their backend infrastructure.
* **API development:** Python's ease of use and libraries like Flask-RESTful and Django REST framework make it ideal for creating RESTful APIs that allow different applications to communicate with each other.
* **Web scraping:** Libraries like Beautiful Soup and Scrapy enable extracting data from websites for various purposes, such as market research, price comparison, and data analysis.

**Data Science and Machine Learning:**

* **Data analysis and visualization:** Libraries like Pandas, NumPy, and Matplotlib provide powerful tools for data manipulation, analysis, and c

## Custom Explanor

In [53]:
from langchain.schema import HumanMessage , AIMessage , SystemMessage

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

customPrompt = "You are a specialized AI model that can answer questions in a very easy way\
    For Example: USER: What is Langchain? AI: Langchain is a python library that helps you to interact with different AI models, Use Cases: XYZ , Big companies that uses it: Blah blah"

llm.invoke([SystemMessage(content=customPrompt) , HumanMessage(content="Who is Fireship?")])


AIMessage(content='Fireship is a popular YouTube channel and online learning platform created by Jeff Delaney.  It focuses on web development tutorials, covering topics like JavaScript frameworks (React, Angular, Vue.js), Firebase, Flutter, and other modern technologies.\n\n**Use Cases:** Learning to code, staying up-to-date with the latest web technologies, building projects.\n\n**Popular for:**  Concise and fast-paced tutorials, high-quality production, and covering trending topics.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c4b9d05e-101b-4e65-a990-e847668eeca4-0', usage_metadata={'input_tokens': 64, 'output_tokens': 97, 'total_tokens': 161, 'input_token_details': {'cache_read': 0}})

## Chaining and Base Output (CONFUSED)

In [54]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)


In [55]:
class CommaSeperatedOutput(BaseOutputParser):
    def parse(self, text):
        return text.strip().split(",")

In [70]:
chatPrompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant gives five different synonyms for each selected word, Seperate the words using commas please."),
    ("human" , "{text}")
])

In [71]:
chain = LLMChain(llm=llm, prompt=chatPrompt , output_parser=CommaSeperatedOutput())

print(chain.run({"text": "intelligent"}))

['bright', ' clever', ' brilliant', ' sharp', ' astute']
